In [1]:
from theano.sandbox import cuda
cuda.use('gpu1')
three_char = False
eight_char = False
sequence_rnn = False 
first_rnn = False
stateful_model = False
theano_rnn = True

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5103)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
path = get_file('nietzche.txt', 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read()
print("Corpus length: %s" % len(text))

Corpus length: 600901


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
chars.insert(0, "\0")

In [5]:
print("Total chars: ", vocab_size)

Total chars:  86


In [6]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [7]:
char_indices = dict((c, i) for (i, c) in enumerate(chars))
indices_char = dict((i, c) for (i, c) in enumerate(chars))

In [8]:
idx = [char_indices[c] for c in text]

In [9]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [10]:
''.join([chars[i] for i in idx[:100]])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all ph'

# 3-Char Model

In [11]:
seq_length = 3
char_0 = [idx[i] for i in range(0, len(idx)-1-seq_length, seq_length)]
char_1 = [idx[i + 1] for i in range(0, len(idx)-1-seq_length, seq_length)]
char_2 = [idx[i + 2] for i in range(0, len(idx)-1-seq_length, seq_length)]
char_3 = [idx[i + 3] for i in range(0, len(idx)-1-seq_length, seq_length)]

In [12]:
x0 = np.array(char_0[:-2])
x1 = np.array(char_1[:-2])
x2 = np.array(char_2[:-2])
x3 = np.array(char_3[:-2])

In [13]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [14]:
n_fac = 42

In [15]:
inp0, emb0 = embedding_input('c0_dat', vocab_size, n_fac)
inp1, emb1 = embedding_input('c1_dat', vocab_size, n_fac)
inp2, emb2 = embedding_input('c2_dat', vocab_size, n_fac)

In [16]:
n_hidden = 256

In [17]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='tanh')
dense_out = Dense(vocab_size, activation='softmax')

In [18]:
# these embs are already linked to the inps from the embedding_input function
hidden = dense_in(emb0)
c2_dense = dense_in(emb1)
hidden = merge([dense_hidden(hidden), c2_dense])
c3_dense = dense_in(emb2)
hidden = merge([dense_hidden(hidden), c3_dense])
c4_out = dense_out(hidden)

In [19]:
model = Model([inp0, inp1, inp2], c4_out)

In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
# model.optimizer.lr = 0.000001
# model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [21]:
if three_char:
    model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [22]:
model.optimizer.lr = 0.01

In [23]:
if three_char:
    model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [24]:
if three_char:
    model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [25]:
model.optimizer.lr = 0.000001

In [26]:
if three_char:
    model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [27]:
if three_char:
    model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [28]:
if three_char:
    model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [29]:
if three_char:
    model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [30]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [31]:
if three_char:
    get_next('the')

In [32]:
if three_char:
    get_next('thi')

In [33]:
if three_char:
    get_next('hel')

In [34]:
if three_char:
    get_next('Som')

In [35]:
if three_char:
    get_next('Wha')

In [36]:
if three_char:
    get_next('is ')

In [37]:
if three_char:
    get_next('s t')

# Our First RNN

In [38]:
cs = 8

In [39]:
c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)] for n in range(cs)]

In [40]:
len(c_in_dat[0])

75112

In [41]:
len(c_in_dat), len(c_in_dat[0])

(8, 75112)

In [42]:
c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]

In [43]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [44]:
len(xs), len(xs[0])

(8, 75110)

In [45]:
y = np.stack(c_out_dat[:-2])

In [46]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [47]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [48]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [49]:
n_fac = 42

In [50]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [51]:
cs, vocab_size, n_fac

(8, 86, 42)

In [52]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [53]:
n_hidden = 256

In [54]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [55]:
hidden = dense_in(c_ins[0][1])

In [56]:
for i in range(1, cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

In [57]:
c_out = dense_out(hidden)

In [58]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [59]:
# model.fit(xs, y, batch_size=64, nb_epoch=12)

In [60]:
# model.fit(xs, y, batch_size=64, nb_epoch=12)

In [61]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [62]:
if eight_char:
    get_next('for thos')

In [63]:
if eight_char:
    get_next('part of ')

In [64]:
if eight_char:
    get_next('queens a')

# Our first RNN

In [65]:
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-cs, cs)] for n in range(cs)]

In [66]:
len(c_out_dat), len(c_out_dat[0])

(8, 75112)

In [67]:
[indices_char[i[0]] for i in c_in_dat]

['P', 'R', 'E', 'F', 'A', 'C', 'E', '\n']

In [68]:
[indices_char[i[0]] for i in c_out_dat]

['R', 'E', 'F', 'A', 'C', 'E', '\n', '\n']

In [69]:
[indices_char[i[1]] for i in c_in_dat]

['\n', '\n', 'S', 'U', 'P', 'P', 'O', 'S']

In [70]:
[indices_char[i[1]] for i in c_out_dat]

['\n', 'S', 'U', 'P', 'P', 'O', 'S', 'I']

In [71]:
ys = [np.stack(arr[:-2]) for arr in c_out_dat]

In [72]:
ys

[array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56]),
 array([ 1, 33,  2, ..., 71, 61, 54])]

In [73]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [74]:
[ys[n][:cs] for n in range(cs)]

[array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67]),
 array([ 1, 33,  2, 72, 67, 73,  2, 68])]

In [75]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [76]:
inp1 = Input(shape=(n_fac,), name = 'zeros')
hidden = dense_in(inp1)

In [77]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = merge([c_dense, dense_hidden(hidden)], mode='sum')
    outs.append(dense_out(hidden)) 

In [78]:
len(outs)

8

In [79]:
model = Model(input=[inp1] + [c[0] for c in c_ins], output=outs)

In [80]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [81]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))

In [82]:
zeros

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [83]:
if first_rnn:
    model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

In [84]:
[zeros] + xs

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [85]:
zeros.shape

(75110, 42)

In [86]:
len(xs), len(xs[0])

(8, 75110)

In [87]:
xs[0]

array([40,  1, 33, ..., 72, 71, 61])

In [88]:
zeros[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.])

In [89]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [90]:
# def get_nexts(inp):
#     # get character indices for input
#     idxs = [char_indices[c] for c in inp]
#     # turn each index into a 1-d array
#     arrs = [np.array([i]) for i in idxs]
#     # predict these arrays, outputting a softmax
#     p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
#     # return the character that maximizes the softmax probability
#     return [chars[np.argmax(o)] for o in p]
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array([i]) for i in idxs]
    p = model.predict([np.reshape(np.zeros(n_fac), newshape=(-1, n_fac))] + arrs)
    return [chars[np.argmax(o)] for o in p]

In [91]:
np.reshape(np.zeros(n_fac), newshape=(-1, n_fac)).shape

(1, 42)

In [92]:
if first_rnn:
    get_nexts(' this is')

In [93]:
if first_rnn:
    get_nexts(' part of')

# Our First Keras RNN

In [94]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

In [95]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=cs),
    SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
    Dense(vocab_size, activation='softmax')
])

In [96]:
c_ins = [embedding_input(name='c_' + str(i), n_in=vocab_size, n_out=n_fac) for i in range(cs)]

In [97]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [98]:
len(idx)

600901

In [99]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [100]:
n_hidden

256

In [101]:
# outs = []
# for i in range(cs):
#     hidden = merge([dense_in(c_ins[i][1]), dense_hidden(hidden)], mode='sum')
#     outs.append(dense_out(hidden)) 

outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    # every layer now has an output
    outs.append(dense_out(hidden))

In [102]:
mdl = Model([inp1] + [c[0] for c in c_ins], outs)

In [103]:
len(c_ins)

8

In [104]:
zs = np.tile(np.zeros(n_fac), (len(xs[0]), 1))

In [105]:
x0 = [np.array([0])] * len(xs[0])

In [106]:
zs.shape

(75110, 42)

In [107]:
mdl.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
if first_rnn:
    mdl.fit([zs]+xs, y, batch_size=64, nb_epoch=12)

# Sequence Model w/ Keras

In [108]:
n_hidden, n_fac, cs, vocab_size

(256, 42, 8, 86)

In [109]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=cs),
    SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

In [110]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_5 (Embedding)          (None, 8, 42)         3612        embedding_input_2[0][0]          
____________________________________________________________________________________________________
simplernn_2 (SimpleRNN)          (None, 8, 256)        76544       embedding_5[0][0]                
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 8, 86)         22102       simplernn_2[0][0]                
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
____________________________________________________________________________________________________


In [111]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [112]:
x_rnn = np.stack(xs, axis=1)

In [113]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [114]:
x_rnn

array([[40, 42, 29, ..., 27, 29,  1],
       [ 1,  1, 43, ..., 40, 39, 43],
       [33, 38, 31, ..., 61, 54, 73],
       ..., 
       [72, 73, 62, ...,  2, 76, 68],
       [71, 65, 57, ..., 72,  2, 73],
       [61, 58,  2, ..., 57, 62, 56]])

In [115]:
y_rnn = np.expand_dims(np.stack(ys, axis=1), -1)

In [116]:
y_rnn.shape

(75110, 8, 1)

In [117]:
if sequence_rnn:
    model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

In [118]:
if sequence_rnn:
    model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

In [119]:
def get_next(inp):
    idx = np.array([char_indices[i] for i in inp])
    arr = idx[np.newaxis, :]
    pred = model.predict(arr)[0]
    return [indices_char[np.argmax(o)] for o in pred]

In [120]:
if sequence_rnn:
    get_next('The thin')

In [121]:
if sequence_rnn:
    get_next(' this is')

In [122]:
model = Sequential([
   SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size), 
            activation='relu', inner_init='identity'),
   TimeDistributed(Dense(vocab_size, activation='softmax'))
])

In [123]:
oh_xs = np.stack([to_categorical(i, vocab_size) for i in xs], 1)

In [124]:
oh_ys = np.stack([to_categorical(i, vocab_size) for i in ys], 1)

In [125]:
oh_xs.shape

(75110, 8, 86)

In [126]:
oh_ys.shape

(75110, 8, 86)

In [127]:
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [128]:
if sequence_rnn:
    model.fit(oh_xs, oh_ys, batch_size=64)

# Stateful Model with Keras

In [129]:
bs = 64

In [130]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs, 8)),
    BatchNormalization(),
    LSTM(n_hidden, return_sequences=True, stateful=True),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

In [131]:
x_rnn.shape

(75110, 8)

In [132]:
y_rnn.shape

(75110, 8, 1)

In [133]:
mx = len(x_rnn)//bs * bs

In [134]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy')

In [135]:
x_rnn

array([[40, 42, 29, ..., 27, 29,  1],
       [ 1,  1, 43, ..., 40, 39, 43],
       [33, 38, 31, ..., 61, 54, 73],
       ..., 
       [72, 73, 62, ...,  2, 76, 68],
       [71, 65, 57, ..., 72,  2, 73],
       [61, 58,  2, ..., 57, 62, 56]])

In [136]:
if stateful_model:
    model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

In [137]:
if stateful_model:
    model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

In [138]:
if stateful_model:
    model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

In [139]:
x_rnn[:mx].shape

(75072, 8)

In [140]:
bs

64

In [141]:
x_rnn.shape

(75110, 8)

In [142]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (64, 8, 42)           3612        embedding_input_3[0][0]          
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (64, 8, 42)           168         embedding_6[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (64, 8, 256)          306176      batchnormalization_1[0][0]       
____________________________________________________________________________________________________
timedistributed_3 (TimeDistribut (64, 8, 86)           22102       lstm_1[0][0]                     
Total params: 332,058
Trainable params: 331,974
Non-trainable params: 84
__________________

In [143]:
x_rnn.shape[1]

8

In [144]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (64, 8, 42)           3612        embedding_input_3[0][0]          
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (64, 8, 42)           168         embedding_6[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (64, 8, 256)          306176      batchnormalization_1[0][0]       
____________________________________________________________________________________________________
timedistributed_3 (TimeDistribut (64, 8, 86)           22102       lstm_1[0][0]                     
Total params: 332,058
Trainable params: 331,974
Non-trainable params: 84
__________________

In [145]:
def print_example():
    seed_string="ethics i"
    batch_size=64
    next_chars = []
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-8:]])[np.newaxis, :]
        remaining_rows = batch_size - x.shape[0]
        newx = np.vstack((x, np.tile(x, (remaining_rows, 1))))
        preds = model.predict(newx, batch_size=batch_size, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        next_chars.append(next_char)
        seed_string = seed_string + next_char
    print(''.join(next_chars))

In [146]:
print_example()

04?;8fmnCa1jVT3e:=FL.gIn7,oBTdHoU![pI-CG:S�KY: ))Tw�M�YwF"q:E�)b"�a3swknz[!t!?V8(jfbH)'WRhyx[2]kg5PF��4nzm�x�4XNmHcwmGL�0])=8uf;P1jita�Elt'1x(soUludzYdFcj,;siVlgv9A:3''w_hn5;kn?hVosVLF1i(YG
 Tya4WcNE�U�oI�'Iz0q?[Dwx1=ePFjW"k9RrB;yD1'oW5D1MKL4:�X)V3]u62,�zoWqN"r_vVt7g'K7?"!mqNjzx,47r0ycI]0Z�Ngt!yH:"h�myT?l3z,r2aH(MP,�


# Theano RNN

In [147]:
n_input = vocab_size
n_output = vocab_size

In [148]:
def init_wgts(rows, cols): 
    scale = math.sqrt(2/rows)
    return shared(normal(scale=scale, size=(rows, cols)).astype(np.float32))
def init_bias(rows): 
    return shared(np.zeros(rows, dtype=np.float32))

In [149]:
def wgts_and_bias(n_in, n_out): 
    return init_wgts(n_in, n_out), init_bias(n_out)
def id_and_bias(n): 
    return shared(np.eye(n, dtype=np.float32)), init_bias(n)

In [150]:
t_inp = T.matrix('inp')
t_outp = T.matrix('outp')
t_h0 = T.vector('h0')
lr = T.scalar('lr')

all_args = [t_h0, t_inp, t_outp, lr]

In [151]:
W_h = id_and_bias(n_hidden)
W_x = wgts_and_bias(n_input, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
w_all = list(chain.from_iterable([W_h, W_x, W_y]))

In [152]:
def step(x, h, W_h, b_h, W_x, b_x, W_y, b_y):
    # Calculate the hidden activations
    h = nnet.relu(T.dot(x, W_x) + b_x + T.dot(h, W_h) + b_h)
    # Calculate the output activations
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    # Return both (the 'Flatten()' is to work around a theano bug)
    return h, T.flatten(y, 1)

In [153]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

In [154]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [155]:
def upd_dict(wgts, grads, lr): 
    return OrderedDict({w: w-g*lr for (w,g) in zip(wgts,grads)})

upd = upd_dict(w_all, g_all, lr)

In [156]:
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [157]:
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

In [158]:
X = oh_x_rnn
Y = oh_y_rnn
X.shape, Y.shape

((75110, 8, 86), (75110, 8, 86))

In [159]:
err = 0.0; l_rate = 0.01
for i in range(len(X)):
    err += fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999:
        print("Error:{:,.3f}".format(err/1000))
        err = 0.0

Error:25.083
Error:21.407
Error:20.868
Error:19.858
Error:18.819
Error:19.220
Error:19.058
Error:18.493
Error:17.940
Error:18.256
Error:17.443
Error:17.652
Error:18.372
Error:17.286
Error:16.790
Error:17.717
Error:17.347
Error:17.177
Error:16.798
Error:16.648
Error:16.502
Error:16.433
Error:16.666
Error:16.176
Error:16.757
Error:16.601
Error:16.077
Error:16.195
Error:16.216
Error:16.410
Error:16.655
Error:16.349
Error:16.676
Error:16.320
Error:15.956
Error:16.728
Error:15.991
Error:16.366
Error:16.057
Error:16.264
Error:15.368
Error:15.713
Error:15.689
Error:15.922
Error:15.918
Error:15.900
Error:15.555
Error:16.095
Error:15.929
Error:16.028
Error:15.249
Error:15.613
Error:14.916
Error:14.876
Error:15.647
Error:15.344
Error:14.679
Error:15.460
Error:15.080
Error:14.985
Error:14.974
Error:15.393
Error:15.296
Error:15.049
Error:14.717
Error:14.782
Error:14.230
Error:14.751
Error:15.175
Error:14.745
Error:15.053
Error:14.670
Error:14.443
Error:14.560
Error:14.426


In [161]:
f_y = theano.function([t_h0, t_inp], v_y, allow_input_downcast=True)

In [162]:
pred = np.argmax(f_y(np.zeros(n_hidden), X[6]), axis=1)

In [163]:
act = np.argmax(X[6], axis=1)

In [164]:
[indices_char[o] for o in act]

['t', 'h', 'e', 'n', '?', ' ', 'I', 's']

In [165]:
[indices_char[o] for o in pred]

['h', 'e', ' ', ' ', ' ', 'T', 't', ' ']